# Build *Bacillus subtilis*

In [ ]:
from __future__ import print_function, division, absolute_import

import sys

# python imports
import re
from os.path import join,dirname,abspath
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from bacillusme.util.helper_functions import *

%load_ext autoreload
%autoreload 2
print(cobrame.__file__)
print(bacillusme.__file__)

In [ ]:
# Define Models
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844_mod.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
ijo = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

In [ ]:
model = cobra.io.load_json_model("./building_data/iYO844.json")
curation = read_curation("./building_data/sporeme_metabolic_curation.csv")
curate_model(model,curation)
model.optimize()
assert model.solution.f > 1e-3

In [ ]:
## M-model Fixes
ijo.reactions.BIOMASS_BS_10.lower_bound = 0
ijo.reactions.BIOMASS_BS_10.upper_bound = 0

## Part 1: Create minimum solveable ME-model
This will include the bare minimum representations of 
- Transcription Reactions
- Translation Reactions
- Complex Formation Reactions
- Metabolic Reactions

that still produce a working ME-model

### 1) Create Model Object and populate its global info
This includes important parameters that are used to calculate coupling 
constraints as well as organism-specific information such as peptide 
processing types

In [ ]:
me = cobrame.MEModel('iJT964-ME')

# ME-models require special OptLang interface if cobrapy version >= 0.6.0
# If cannot import SymbolicParameter, assume using cobrapy
# versions <= 0.5.11
try:
    from optlang.interface import SymbolicParameter
except:
    pass
else:
    me.solver = me_model_interface

# "Translational capacity" of organism
me.global_info['kt'] = 4.5  # (in h-1)scott 2010, RNA-to-protein curve fit
me.global_info['r0'] = 0.087  # scott 2010, RNA-to-protein curve fit
me.global_info['k_deg'] = 1.0/5. * 60.0  # 1/5 1/min 60 min/h # h-1

# Molecular mass of RNA component of ribosome
me.global_info['m_rr'] = 1453. # in kDa

# Average molecular mass of an amino acid
me.global_info['m_aa'] = 109. / 1000.  # in kDa

# Proportion of RNA that is rRNA
me.global_info['f_rRNA'] = .86
me.global_info['m_nt'] = 324. / 1000.  # in kDa
me.global_info['f_mRNA'] = .02

# tRNA associated global information
me.global_info['m_tRNA'] = 25000. / 1000.  # in kDA
me.global_info['f_tRNA'] = .12

# Folding Properties
me.global_info['temperature'] = 37
me.global_info['propensity_scaling'] = .45

# DNA Replication Parameters
me.global_info['GC_fraction'] = 0.507896997096

# Define the types of biomass that will be synthesized in the model
me.add_biomass_constraints_to_model(["protein_biomass", "mRNA_biomass", "tRNA_biomass", "rRNA_biomass",
                                     "ncRNA_biomass", "DNA_biomass", "lipid_biomass", "constituent_biomass",
                                     "prosthetic_group_biomass", "peptidoglycan_biomass"])

# Define ME-model compartments
me.compartments = {"p": "Periplasm", "e": "Extra-organism", "c": "Cytosol", 
                   "im": 'Inner Membrane', 'om': "Outer Membrane", "mc": "ME-model Constraint",
                   "m": "Membrane"}

### 2) Load metabolites and build Metabolic reactions
The below reads in:
- Required 
 * **reaction_matrix.txt** (reaction matrix w/ reactions unlumped, metabolites renamed etc.)
 * **metabolites.txt** (metabolite properties)
 * **reactions.txt** (info on reversiblity, whether enzyme catalyzed etc.)
 * **m_to_me_mets.csv** (mapping of enzymes/complexes used in M-model to their ME-model compatible ID)
 * **protein_complexes.txt** (protein subunit stoichiometry of all complexes, used to identify metabolites as such)
 
It creates a new M-model from this info then incorporates it into the ME-model using *add_m_model_content*. metabolites are added directly reactions are added as StoichiometricData

Metabolite types have different properties in an ME-model so enzyme complexes need added to the model as Complexes not Metabolites. Components in the M-model that are actually Complexes are compiled in *complex_list*

In [ ]:
# m_model = flat_files.get_m_model()
m_model = flat_files.process_m_model(ijo, 'metabolites.txt',
                                     'm_to_me_mets.csv', 'reactions.txt',
                                     'reaction_matrix.txt',
                                     'protein_complexes.txt',
                                     defer_to_rxn_matrix=set())
m_model.reactions.EX_glc__D_e.id = 'EX_glc__D_e'
m_model.repair()
# Manual curation Oct 2023
curation = read_curation("./building_data/sporeme_metabolic_curation.csv")
curate_model(m_model,curation)

# some of the "metabolites" in iJO1366 "M" model are actually complexes. We pass those in
# so they get created as complexes, not metabolites.
complexes = flat_files.get_complex_subunit_stoichiometry(
    'protein_complexes.txt').keys()
complex_set = set(
    [i.id for i in m_model.metabolites if i.id.split('_mod_')[0] in complexes])
building.add_m_model_content(me, m_model, complex_metabolite_ids=complex_set)

In [ ]:
# This adds exchange reactions for metabolites not contained in iJO1366
# Some of these cannot be produced by the model so they are added here
exchange_list = ['4fe4s_c','2fe2s_c','udcpp_c','pydx5p_c','3fe4s_c','cl_c','sheme_c','cu_c','mn2_c',
                 'bmocogdp_c','dpm_c','thmpp_c','zn2_c','cbl1_c','cobalt2_c']
transport_candidates = {}
bsub_ecoli_blast = pd.read_csv('./building_data/blast_out_bsub_as_ref.txt',sep='\t',index_col=2)
for met_id in exchange_list:   
    # Identify or create metabolites
    if hasattr(me.metabolites,met_id):
        met = me.metabolites.get_by_id(met_id)
    else:
        met = cobrame.Metabolite(met_id)
    met_e_id = re.sub('_.','_e',met_id)
    if hasattr(me.metabolites,met_e_id):
        met_e = me.metabolites.get_by_id(met_e_id)
    else:
        met_e = cobrame.Metabolite(met_e_id)
        
    # Identify or create exchange reaction
    ex_id = 'EX_{}'.format(met_e_id)
    if hasattr(me.reactions,ex_id):
        ex_r = me.reactions.get_by_id(ex_id)
    else:
        ex_r = cobrame.MEReaction(ex_id)
        ex_r.add_metabolites({met_e:-1})
        me.add_reaction(ex_r)
    ex_r.upper_bound = 0 # Avoid secretion of this metabolites. These reactions are
                         # only meant to supply cofactors for which no biosynthetic 
                         # pathway has been defined in the model.
    ex_r.lower_bound = -1000
    
    # Identify or create transport reaction. Available transport reactions in model?
    met_prod_rxns = [rxn.id for rxn in get_reactions_of_met(m_model,met.id,s=1,verbose=0)]
    met_e_cons_rxns = [rxn.id for rxn in get_reactions_of_met(m_model,met_e.id,s=-1,verbose=0)]
    transport = list(set(met_prod_rxns)&set(met_e_cons_rxns))
    ref = 'none'
    if transport:
        ref = 'bsub'
        for r in transport:
            rxn = m_model.reactions.get_by_id(r)
            rxn.lower_bound = 0
            rxn.upper_bound = 1000
            print('Found in model:',met_id,r,rxn.reaction)
    else:
        # In E. coli as template?
        transporters = []
        met_prod_rxns = [rxn.id for rxn in get_reactions_of_met(eco,re.sub('_.','_p',met.id),s=1,verbose=0)]
        met_e_cons_rxns = [rxn.id for rxn in get_reactions_of_met(eco,met_e.id,s=-1,verbose=0)]
        transport = list(set(met_prod_rxns)&set(met_e_cons_rxns))
        if transport:
            ref = 'eco'
            for r in transport:
                rxn = eco.reactions.get_by_id(r)
                transporters = [bsub_ecoli_blast.loc[gene.id]['BSUB_gene'] for gene in rxn.genes \
                                    if gene.id in bsub_ecoli_blast.index]
                if transporters:
                    transport_candidates[met_id] = transporters
                    print('Manually curate: ', met_id, transport)
                else:
                    art = 1
        else:
            art = 1
        if art:
            ## Add artificial reaction
            t_r = cobrame.MEReaction(met_id.split('_')[0].upper()+'t')
            t_r.add_metabolites({met_e:-1,met:1})
            print('Artificially added: ', met_id)
            me.add_reaction(t_r)
    #_ = get_reactions_of_met(me,met_e.id)

### 3) Add Transcription and Translation 
The below reads in:
- Required
 * **NC_000913.2.gb** (Genbank sequence annotation)
 * **ecolime/translation.py** (codon to tRNA mapping)
- Optional
 * **TUs_from_ecocyc.txt** (TU definitions, start/stop positions, etc.)
 * **ecolime/translation.py** (dictionary of gene to frameshift mutation)
 
To construct the bare minimimum components of a transcription and translation reactions. For example, transcription reactions at this point include nucleotides and the synthesized RNAs.

In [ ]:
element_types = {'CDS', 'rRNA','tRNA', 'ncRNA','misc_RNA'}

In [ ]:
gb_filename = join(flat_files.ecoli_files_dir,'NC_000964.gb')                                                                                    
tu_df = flat_files.get_tu_dataframe('TUs_from_bsubcyc.txt')

building.build_reactions_from_genbank(me, gb_filename, tu_df, verbose=False,
                                      frameshift_dict=translation.frameshift_dict,
                                      trna_to_codon=translation.trna_to_codon,
                                      element_types = element_types)



### 4) Add in complex Formation without modifications (for now)

The below reads in:
- Required
 * **protein_complexes.txt** (Metabolic complexes' protein subunit stoichiometries)
 * **protein_modification.txt** (Type and number of modifications for each protein)

In [ ]:
# complex_stoichiometry_dict is a dict of {'complex_id': [{'bnum' : count}]}
rna_components = {'BSU_misc_RNA_2','BSU_misc_RNA_35'} # component id should have 'RNA_ instead' of 'protein_'
# get Metabolic Complex composition from ECOLIme
complex_stoichiometry_dict = flat_files.get_complex_subunit_stoichiometry('protein_complexes.txt', rna_components)
# add complexes to model
complex_modification_dict = flat_files.get_complex_modifications('protein_modification.txt', 'protein_complexes.txt')
building.add_model_complexes(me, complex_stoichiometry_dict, complex_modification_dict, verbose=False)

# remove modifications. they will be added back in later
for data in me.complex_data:
    data.subreactions = {}
# add formation reactions for each of the ComplexDatas
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

### 5) Add dummy reaction to model and unmodeled_protein_fraction

Includes the transcription, translation, complex_formation, and metabolic reaction. Sequence based on prevelance of each codon found.
 - Required
     * [**codon_usage.csv**](http://openwetware.org/wiki/Escherichia_coli/Codon_usage) (codon prevelance)

In [ ]:
seq = "ATG"
codons = pandas.read_csv(join(flat_files.ecoli_files_dir, "codon_usage.csv"), index_col=1)
for codon, row in codons.iterrows():
    if row.amino_acid == "Stop":
        continue
    seq += codon * int(row.per_1000 // 3)  # want roughly 300 aa
# get the most used stop codon
seq += codons[codons.amino_acid == "Stop"].sort_values("per_1000").index[-1]
building.add_dummy_reactions(me, seq, update=True)

# dummy proteins can be used to catalyze orphan reactions. This
# ensures that orphan reactions will not becore favored to fulfil 
# unmodeled protein fraction requirement 
rxn = cobrame.SummaryVariable('dummy_protein_to_mass')
me.add_reactions([rxn])
mass = me.metabolites.protein_dummy.formula_weight / 1000.  # in kDa
rxn.add_metabolites({'protein_biomass': -mass, 'protein_dummy': -1,
                     cobrame.Constraint('unmodeled_protein_biomass'): mass})

In [ ]:
get_reactions_of_met(me,'unmodeled_protein_biomass')

In [ ]:
for r in me.reactions:
    if 'biomass' in r.id:
        print(r.id, r.reaction , r.bounds)

In [ ]:
for r in me.metabolites.unmodeled_protein_biomass.reactions:
    print(r.id,r.reaction)

In [ ]:
for r in me.metabolites.protein_dummy.reactions:
    print(r.id,r.reaction)

### 6) Assocated Complexes and build Metabolic Reactions 
- Required
    * **enzyme_reaction_association.txt**
    * **reactions.txt** (gives reaction name, reversiblity, source and whether reaction is spontaneous)


In [ ]:
# associate reaction id with the old ME complex id (including modifications)
rxn_to_cplx_dict = flat_files.get_reaction_to_complex(m_model)
rxn_info = flat_files.get_reaction_info_frame('reactions.txt')

# Required to add dummy reaction as spontaneous reaction
rxn_info = rxn_info.append(pandas.Series({'description': 'dummy reaction', 'is_reversible': 0,
                                          'is_spontaneous':1}, name='dummy_reaction'))

building.add_reactions_from_stoichiometric_data(me, rxn_to_cplx_dict, rxn_info, update=True)

### 7) Incorporate remaining biomass constituents
There are leftover components from the biomass equation that either: 
1. have no mechanistic function in the model (glycogen)
2. are cofactors that are regenerated (nad)

Applies demands and coefficients from the biomass objective function

In [ ]:
me.ngam = 9.
me.gam = 105.053483
me.unmodeled_protein_fraction = .36

biomass_constituents = {
    "10fthf_c": -0.000367,
    "peptido_BS_c": -.101817,
    "psetha_BS_c": -0.00056,
    "mql7_c":-0.000266,
}


rxn = cobrame.SummaryVariable('biomass_constituent_demand')
me.add_reactions([rxn])
rxn.add_metabolites(biomass_constituents)
constituent_mass = sum(me.metabolites.get_by_id(c).formula_weight / 1000. * -v
                     for c, v in biomass_constituents.items())
rxn.lower_bound = mu
rxn.upper_bound = mu
rxn.add_metabolites({me.metabolites.constituent_biomass: constituent_mass})

 #### Lipid components
 Metabolites and coefficients from biomass objective function

In [ ]:
# Identify lipid metabolites in biomass equation

lipid_demand = {}
for key, value in ijo.reactions.BIOMASS_BS_10.metabolites.items():
    if '_BS_c' in key.id and not key.id in biomass_constituents:
        lipid_demand[key.id] = abs(value)
                
for met, requirement in lipid_demand.items():
    component_mass = me.metabolites.get_by_id(met).formula_weight / 1000.
    rxn = cobrame.SummaryVariable('Demand_' + met)
    me.add_reactions([rxn])
    rxn.add_metabolites({met: -1 * requirement,
                         'lipid_biomass': component_mass * requirement})
    rxn.lower_bound = mu
    rxn.upper_bound = 1000.

# Kdo2lipid4
#requirement = 0.01945  # in mmol/gDW
#met = me.metabolites.get_by_id('kdo2lipid4_e')
#component_mass = met.formula_weight / 1000.
#rxn = cobrame.SummaryVariable('Demand_' + met.id)
#me.add_reactions([rxn])

#rxn.add_metabolites({met.id : -1.*requirement,
#                     'lipid_biomass': component_mass * requirement})
#rxn.lower_bound = mu
#rxn.upper_bound = mu

#### DNA Demand Requirements
Added based on growth rate dependent DNA levels as in [O'brien EJ et al 2013](https://www.ncbi.nlm.nih.gov/pubmed/24084808)

In [ ]:
dna_demand_stoich, dna_demand_bound = bacillusme.dna_replication.return_gr_dependent_dna_demand(
    me, me.global_info['GC_fraction'])

dna_replication = cobrame.SummaryVariable("DNA_replication")
me.add_reaction(dna_replication)
dna_replication.add_metabolites(dna_demand_stoich)
dna_biomass = cobrame.Constraint("DNA_biomass")

dna_mw = 0
dna_mw_no_ppi = bacillusme.dna_replication.get_dna_mw_no_ppi_dict(me)
for met, value in me.reactions.DNA_replication.metabolites.items():
    if met.id != 'ppi_c':
        dna_mw -= value * dna_mw_no_ppi[met.id.replace('_c','')] / 1000.
        
dna_replication.add_metabolites({dna_biomass: dna_mw})
dna_replication.lower_bound = dna_demand_bound 
dna_replication.upper_bound = dna_demand_bound 


**Note**: From this point forward, executing every codeblock should result in a solveable ME-model

------

## Part 2: Add metastructures to solving ME-model
This includes:
1. ribosome
2. RNA polymerase
3. charged_tRNAs

Sometimes multiple entities can perform the same role. To prevent a combinatorial explosion of possibilities, we can create  "generic" version, where any of those entities can fill in.

In [ ]:
for generic, components in generics.generic_dict.items():
    cobrame.GenericData(generic, me, components).create_reactions()

### 1) Add ribosome
This uses the ribosome composition definition in **ecolime/ribosome.py**

In [ ]:
bacillusme.ribosome.add_ribosome(me, verbose=False)

### 2) Add charged tRNA reactions

The tRNA charging reactions were automatically added when loading the genome from the genbank file. However, the charging reactions still need to be made aware of the tRNA synthetases which are responsible.

Uses **trna_charging.py**

In [ ]:
aa_synthetase_dict = bacillusme.trna_charging.amino_acid_trna_synthetase
for data in me.tRNA_data:
    data.synthetase = str(aa_synthetase_dict[data.amino_acid])

Generic charged tRNAs are added to translation reactions via SubreactionData below.

All new data added in this block contained in **ecolime/translation.py**

In [ ]:
bacillusme.translation.add_charged_trna_subreactions(me)
for data in me.translation_data:
    data.add_initiation_subreactions(start_codons=translation.translation_start_codons,
                                    start_subreactions=set(translation.initiation_subreactions.keys()))

    data.add_elongation_subreactions(elongation_subreactions=set(translation.elongation_subreactions.keys()))

    data.add_termination_subreactions(translation_terminator_dict=translation.translation_stop_dict)

### 3) Add Transcription Metacomplexes
#### RNA Polymerase

Data for RNA_polymerase composition fround in **ecolime/transcription**

Uses *tu_df* from **TUs_from_ecocyc.txt**, above

In [ ]:
for met in transcription.rna_polymerases:
    rnap_obj = cobrame.RNAP(met)
    me.add_metabolites(rnap_obj)
transcription.add_rna_polymerase_complexes(me, verbose=False)

# associate the correct RNA_polymerase and factors to TUs
sigma_to_rnap_dict = transcription.sigma_factor_complex_to_rna_polymerase_dict
for tu_id in tu_df.index:
    transcription_data = me.process_data.get_by_id(tu_id)
    sigma = tu_df.sigma[tu_id]
    rna_polymerase = sigma_to_rnap_dict[sigma]
    transcription_data.RNA_polymerase = rna_polymerase

#### Degradosome (both for RNA degradation and RNA splicing)

All new data contained in **transcription.py**

In [ ]:
me.add_metabolites([cobrame.Complex('RNA_degradosome')])
data = cobrame.ComplexData('RNA_degradosome', me)
for subunit, value in transcription.rna_degradosome.items():
    data.stoichiometry[subunit] = value
data.create_complex_formation(verbose=False)

# Used for RNA splicing
data = cobrame.SubreactionData('RNA_degradation_machine', me)
data.enzyme = 'RNA_degradosome'

data = cobrame.SubreactionData('RNA_degradation_atp_requirement', me)
# .25 water equivaltent for atp hydrolysis per nucleotide
data.stoichiometry = {'atp_c': -.25, 'h2o_c': -.25, 'adp_c': .25,
                      'pi_c': .25, 'h_c': .25}

transcription.add_rna_splicing(me)

------
## Part 3: Add remaining modifications
rRNA modifications handled in *add_ribosome*

### 1) Add complex modifications
*complex_modification_dict* from **protein_modification.text**, above

The rest of the new data contained in **ecolime/modifications.py**

In [ ]:
for complex_id, info in complex_modification_dict.items():
    modifications = {}
    for mod, value in info['modifications'].items():
        # stoichiometry of modification determined in 
        # subreaction_data.stoichiometry
        
        modifications['mod_' + mod] = abs(value)
    me.process_data.get_by_id(complex_id).subreactions = modifications

# Adds modification data for more complicated enzyme modifications
# (ie, iron sulfur cluster modification)
bacillusme.modifications.add_modification_procedures(me)

In [ ]:
with open("./me_models/iBS_checkpoint.pickle", "wb") as outfile:
    pickle.dump(me, outfile)

In [ ]:
with open("./me_models/iBS_checkpoint.pickle", "rb") as outfile:
    me = pickle.load(outfile)

In [ ]:
# add formation reactions for each of the ComplexDatas
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if 'Oxidized' not in cplx_data.id: 
        if formation:
            formation.update()
        else:
            cplx_data.create_complex_formation()

### 2) Add tRNA mods and asocciate them with tRNA charging reactions
New data from:
1. **ecolime/trna_charging.py** (read via *add_tRNA_modification_procedures()*)
2. **post_transcriptional_modification_of_tRNA.txt** (modification types per tRNA)


In [ ]:
# Add tRNA modifications to ME-model
bacillusme.trna_charging.add_trna_modification_procedures(me)

# trna_modifications = {tRNA_id: {modifications: count}}
trna_modifications=flat_files.get_trna_modification_targets()
for trna in trna_modifications:
    for data in me.process_data.query(trna):
        data.subreactions = trna_modifications[trna]

---
## Part 4: Add remaining subreactions
### 1) Add translation related subreactions
All new data from **ecolime/translation.py**

In [ ]:
# add the translation subreaction data objects to model
translation.add_translation_subreactions_to_model(me)

# add translation subreaction data to reactions
methionine_cleaved=translation.methionine_cleaved
folding_dict=translation.folding_dict

for data in me.translation_data:
    locus_id = data.id
    if locus_id in methionine_cleaved:
        data.subreactions['N_terminal_methionine_cleavage'] = 1

    for folding_type in folding_dict:
        if locus_id in folding_dict[folding_type]:
            data.subreactions[folding_type] = 1

    # This block was ran above, but should be ran again to 
    # incorporate any subreactions not added previously
    data.add_initiation_subreactions(
        start_codons=translation.translation_start_codons,
        start_subreactions=set(translation.initiation_subreactions.keys()))
    data.add_elongation_subreactions(
        elongation_subreactions=set(translation.elongation_subreactions.keys()))
    data.add_termination_subreactions(
        translation_terminator_dict=translation.translation_stop_dict)

    # add organism specific subreactions associated with peptide processing
    for subrxn in translation.peptide_processing_subreactions:
        data.subreactions[subrxn] = 1

### 2) Add transcription related subreactions
All new data from **ecolime/transcription.py**

In [ ]:
for subreaction in transcription.transcription_subreactions:
    subreaction_data = cobrame.SubreactionData(subreaction, me)
    enzymes = transcription.transcription_subreactions[subreaction]['enzymes']
    subreaction_data.stoichiometry = transcription.transcription_subreactions[subreaction]['stoich']
    subreaction_data.enzyme = enzymes

for transcription_data in me.transcription_data:
    # Assume false if not in tu_df
    rho_dependent = tu_df.rho_dependent.get(transcription_data.id, False)
    rho = 'dependent' if rho_dependent else 'independent'
    stable = 'stable' if transcription_data.codes_stable_rna else 'normal'
    transcription_data.subreactions['Transcription_%s_rho_%s' % (stable,
                                                                 rho)] = 1

----
## Part 5: Add in translocation

New data from:
1. **peptide_compartment_and_pathways.txt** (Protein compartment and translocation pathway for each membrane complex)
2. **ecolime/translocation.py** (definitions of each translocation pathway)

In [ ]:
# Add TranslocationData
transloc = pandas.read_csv(join(flat_files.ecoli_files_dir,"peptide_compartment_and_pathways.txt"),
                           sep='\t', comment="#")
for pathway, info in bacillusme.translocation.pathway.items():
    if 'alt' not in pathway:
        transloc_data = cobrame.TranslocationData(pathway + '_translocation', me)
    else:
        transloc_data = cobrame.TranslocationData(pathway.replace('_alt', '_translocation_alt'), me)
    transloc_data.enzyme_dict = info['enzymes']
    transloc_data.keff = info['keff']
    transloc_data.length_dependent_energy = info['length_dependent_energy']
    transloc_data.stoichiometry = info['stoichiometry']

# Associate data and add translocation reactions
bacillusme.translocation.add_translocation_pathways(me, transloc, membrane_constraints = False)

# Update stoichiometry of membrane complexes
# new_stoich = {complex_id: protein_w_compartment}
new_stoich = defaultdict(dict)
for cplx, row in transloc.set_index('Complex').iterrows():
    if cplx == 'EG10544-MONOMER':
        continue
    protein = row.Protein.split('(')[0] + '_' + row.Protein_compartment
    value = me.process_data.get_by_id(cplx).stoichiometry['protein_' + row.Protein.split('(')[0]]
    new_stoich[cplx]['protein_' + protein] = float(value)

for cplx, stoich in new_stoich.items():
    complex_data = me.process_data.get_by_id(cplx)
    for met, value in stoich.items():
        complex_data.stoichiometry.pop(met[0:16])
        complex_data.stoichiometry[met] = value
        complex_data.formation.update()
    # Complex ids in protein compartment file doesn't include mods
    # Some have multiple alternative modifications so must loop through these
    for complex_data in me.process_data.query(cplx + '_mod_'):
        for met, value in stoich.items():
            complex_data.stoichiometry.pop(met[0:16])
            complex_data.stoichiometry[met] = value
            complex_data.formation.update()

---
## Part 6: Add Cell Wall Components
All new data from **ecolime/translocation.py**

In [ ]:
compartment_dict = {}
for prot, compartment in transloc.set_index('Protein').Protein_compartment.to_dict().items():
    compartment_dict[prot.split('(')[0]] = compartment

#### Add lipid modification SubreactionData

In [ ]:
# Bsub m-model doest not have pe and pg metabolites

#### Braun's lipoprotein demand
Metabolites and coefficients as defined in [Liu et al 2014](http://bmcsystbiol.biomedcentral.com/articles/10.1186/s12918-014-0110-6)

#### Correct complex formation IDs if they contain lipoproteins

In [ ]:
# Bsub m-model doest not have murein

-----
## Part 7: Set keffs

Either entirely based on SASA or using fit keffs from [Ebrahim et al 2016](https://www.ncbi.nlm.nih.gov/pubmed/27782110?dopt=Abstract)

In [ ]:
# # Set keffs to sasa fluxes centered around 65.
# me.set_SASA_keffs(65)

In [ ]:
## Get median sasa
from cobrame.core.component import Complex
import numpy as np

sasa_list = []
for met in me.metabolites:
    cplx_sasa = 0.
    if not isinstance(met, Complex):
        continue
    MW = met.formula_weight
    if not MW:
        MW = 0
        print(met.id, ' has no formula')
    cplx_sasa += MW ** (3. / 4)
    sasa_list.append(cplx_sasa)
median_sasa = np.median(np.array(sasa_list))

In [ ]:
reaction_median_keffs = pandas.read_csv(flat_files.fixpath('reaction_median_keffs.txt'), sep='\t',
                              names=['Reaction', 'Complex', 'Median_Keff'])
keff_list = []
for key, row in reaction_median_keffs.iterrows():
    reaction_id = row['Reaction']
    complex_id = row['Complex']
    median_keff = row['Median_Keff']
    
    real_reaction_id = reaction_id + ''
    
    molecular_weight = me.metabolites.get_by_id(complex_id).formula_weight
    sasa = molecular_weight ** (3./4)
    keff = sasa * median_keff / median_sasa
    
    if keff > 3000:
        keff = 3000.
    elif keff < .01:
        keff = .01
    keff_list.append(keff)
    try:
        me.reactions.get_by_id(reaction_id).keff = keff
        me.reactions.get_by_id(reaction_id).update()
    except:
        continue
    

-----
## Part 8: Model updates and corrections

In [ ]:
# Add reaction subsystems to model
for rxn in ijo.reactions:
    if rxn.id in me.process_data:
        data = me.process_data.get_by_id(rxn.id)
    else:
        continue
    for r in data.parent_reactions:
        r.subsystem = rxn.subsystem

#### Add enzymatic coupling for "carriers"
These are enzyme complexes that act as metabolites in a metabolic reaction (i.e. are metabolites in iJO1366)

In [ ]:
for data in me.stoichiometric_data:
    if data.id == 'dummy_reaction':
        continue
        
    for met, value in data.stoichiometry.items():
        if not isinstance(me.metabolites.get_by_id(met), cobrame.Complex) or value > 0:
            continue
            
        subreaction_id = met + '_carrier_activity'
        if subreaction_id not in me.process_data:           
            sub = cobrame.SubreactionData(met + '_carrier_activity', me)
            sub.enzyme = met
        data.subreactions[subreaction_id] = abs(value)

----
## Part 9: Update and save

In [ ]:
me.reactions.biomass_dilution.objective_coefficient = 1.
me.reactions.EX_glc__D_e.lower_bound = -1000
me.reactions.EX_o2_e.lower_bound = -1000
me.ngam = 9.
me.gam = 105.053483
me.unmodeled_protein_fraction = .36

me.reactions.BIOMASS_BS_10_FWD_CPLX_dummy.lower_bound = 0
me.reactions.BIOMASS_BS_10_FWD_CPLX_dummy.upper_bound = 0
me.reactions.ATPM_FWD_CPLX_dummy.lower_bound = 0
me.reactions.ATPM_FWD_CPLX_dummy.upper_bound = 0

me.metabolites.get_by_id('mn2_c').formula = 'Mn2'
me.metabolites.get_by_id('mg2_c').formula = 'Mg2'
me.metabolites.get_by_id('ca2_c').formula = 'Ca2'
me.metabolites.get_by_id('cu2_c').formula = 'Cu2'

In [ ]:
me.update()

In [ ]:
me.prune()

### Add remaining complex formulas and compartments to model

In [ ]:
# Update a second time to incorporate all of the metabolite formulas corectly
for r in me.reactions.query('formation_'):
    r.update()

# Update complex formulas
df = pandas.read_table(join(flat_files.ecoli_files_dir, 'modification.txt'),
                            names=['mod', 'formula','na'])
df = df.drop('na', axis=1).set_index('mod').dropna(how='any')
modification_formulas = df.T.to_dict()
formulas.add_remaining_complex_formulas(me, modification_formulas)

# Update reactions affected by formula update 
for r in me.reactions.query('_mod_lipo'):
    r.update()
for r in me.reactions.query('_mod_glycyl'):
    r.update()
    
# add metabolite compartments
compartments.add_compartments_to_model(me)

### Remaining fields

In [ ]:
##
current_gene_ids = [gene.id for gene in me.genes]
for met in me.metabolites:
    met_id = met.id
    if 'protein_BSU' in met_id:
        gene_id = met_id.split('_')[1]
        if gene_id and gene_id not in current_gene_ids:
            try:
                print(gene_id)
                gene = cobra.Gene(gene_id)
                me.genes.append(gene)
            except:
                pass

##
me.reactions.BIOMASS_BS_10_FWD_CPLX_dummy.lower_bound = 0
me.reactions.BIOMASS_BS_10_FWD_CPLX_dummy.upper_bound = 0
me.reactions.ATPM_FWD_CPLX_dummy.lower_bound = 0
me.reactions.ATPM_FWD_CPLX_dummy.upper_bound = 0

## Homogenize exchange reactions
for rxn in me.reactions:
    if 'EX_' in rxn.id:
        try:
            lb = ijo.reactions.get_by_id(rxn.id).lower_bound
            ub = ijo.reactions.get_by_id(rxn.id).upper_bound
            rxn.lower_bound = lb
            rxn.upper_bound = ub
        except:
            print(rxn.id, 'not in m_model')
            
## These exchange reactions need to be opened
mets = ['mn2_e','zn2_e','cobalt2_e']
for met_id in mets:
    r = me.reactions.get_by_id('EX_'+met_id)
    r.lower_bound = -1000
    r.upper_bound = 0

## New fixes

In [ ]:
# Feb 12, 2020
# Model was synthesizing extra lipids to compensate and reach maximum growth rate.
# Fix: Set lb = ub = mass fraction of lipids
rxns = get_reactions_of_met(me,'lipid_biomass', s=1, verbose=False)
for rxn in rxns:
    rxn.upper_bound = mu
_=get_reactions_of_met(me,'lipid_biomass', s=1, verbose=True)

In [ ]:
# Fix names
for r in me.reactions:
    if isinstance(r,cobrame.MetabolicReaction):
        if r.stoichiometric_data.id not in ijo.reactions: continue
        og_r = ijo.reactions.get_by_id(r.stoichiometric_data.id)
        r.name = og_r.name
        r.subsystem = og_r.subsystem

## Save

In [ ]:
solve_me_model(me, max_mu = 0.5, min_mu = .05, using_soplex=False, precision = 1e-6)

In [ ]:
# Save. When ready, copy to me_models/final/
with open("./me_models/solution.pickle", "wb") as outfile:
    pickle.dump(me, outfile)

### Summary info

In [ ]:
from tqdm import tqdm
d = {}
for r in tqdm(me.reactions):
    if r.id in d: continue
    dr = {}
    dr['name'] = r.name
    dr['subsystem'] = r.subsystem
    try:
        dr['reaction'] = r.reaction
    except:
        print('Cant write reaction of ' + r.id)
    if hasattr(r,'complex_data'):
        if r.complex_data:
            cplx = r.complex_data.id.split('_mod_')[0]
            dr['complex'] = cplx
            try:
                dr['genes'] = df[2][cplx]
            except:
                print(r.complex_data.id + ' not in protein_complexes.txt')
    d[r.id] = dr
info_df = pd.DataFrame.from_dict(d).T
info_df.fillna('').to_csv('summary_info.csv')

### Breakdown

In [ ]:
def get_breakdown(me,field):
    count = {}
    for instance in getattr(me,field):
        instance_type = instance.__class__.__name__
        if instance_type not in count.keys():
            count[instance_type] = 0
        count[instance_type] += 1
    return pd.DataFrame.from_dict({'count':count})

In [ ]:
import matplotlib.pyplot as plt
fields = ['metabolites','reactions']

lims = np.array([[0,300,800,1100],[0,1000,1300,1800]])

fig,ax = plt.subplots(2,len(fields),sharex=False)
plt.rcParams['svg.fonttype'] = 'none'
for idx,f in enumerate(fields):
    df = get_breakdown(me,f).sort_values(by='count')
    
    df.plot.bar(legend=False,figsize=(13,5),ax=ax[0,idx])
    df.plot.bar(legend=False,figsize=(13,5),ax=ax[1,idx])
    
    ax[0,idx].set_ylim(lims[idx,2], lims[idx,3])  # outliers only
    ax[1,idx].set_ylim(lims[idx,0], lims[idx,1])  # most of the data
    
    ax[0,idx].spines['bottom'].set_visible(False)
    ax[1,idx].spines['top'].set_visible(False)
    ax[0,idx].tick_params(axis='x', which='both',bottom=False)
    
    ax[0,idx].tick_params(labeltop=False)  # don't put tick labels at the top
    ax[1,idx].xaxis.tick_bottom()
    
    
    ax[0,idx].set_ylabel('Count')
    ax[0,idx].set_title(f)


In [ ]:
# Define Models
temp = join(flat_files.ecoli_files_dir, 'iYO844.json')
clean_bsub = cobra.io.load_json_model(temp)

In [ ]:
me_dict = me.get_metabolic_flux()
clean_bsub.optimize()
m_dict = clean_bsub.solution.x_dict

comparison_dict = {}
for r in me_dict:
    if r not in m_dict:
        print('Skipping {}'.format(r))
        continue
    comparison_dict[r] = {}
    comparison_dict[r]['m'] = m_dict[r]
    comparison_dict[r]['me'] = me_dict[r]

In [ ]:
df = pd.DataFrame.from_dict(comparison_dict).T

In [ ]:
np.log10(df[df.abs()>0]).plot(x='m',y='me',style='o',figsize=(6.5,5))
plt.plot([-6,2],[-6,2],'--k')

plt.xlim([-6.5,6.5])
plt.ylabel('ME-predictions')
plt.xlabel('Mm-predictions')


In [ ]:
fig = plt.figure(figsize=(6.5,5))
np.log10(df[df.abs()>0])['m'].hist(bins=50,alpha=0.5,label='m')
np.log10(df[df.abs()>0])['me'].hist(bins=40,alpha=0.5,label='me')

plt.xlabel('log10(flux)')
plt.ylabel('Count')
plt.legend()
